# Hilti Evaluation wrt GT

In [2]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d 

import pandas as pd
import os
import pprint

from evo.core.trajectory import PoseTrajectory3D
from evo.tools import plot
from evo.core import metrics
from evo.core import sync

from pose_trajectory_evaluation import PoseTrajectoryEvaluation

%reload_ext autoreload
%autoreload 2

In [23]:
export_paths = ['/media/berlukas/Data/data/datasets/Hilti/Lab/']
exported_missions = ['53ab6674092fa0161000000000000000']
eval = PoseTrajectoryEvaluation()

est_trajectories = []
gt_trajectories = []

assert len(export_paths) == len(exported_missions)
for export_path, mission_id in zip(export_paths, exported_missions):
    est_traj_filename = export_path + 'vertex_poses_velocities_biases.csv'
    gt_traj_filename = export_path + 'gt.txt'

    est_df = eval.get_mission_from_csv(est_traj_filename, mission_id)
    est_trajectories.append(eval.convert_df_to_traj(est_df))
    
    gt_df = eval.parse_trajectory_txt(gt_traj_filename)
    gt_trajectories.append(eval.convert_df_to_traj(gt_df))
    
assert len(gt_trajectories) == len(est_trajectories)
print(f'Loaded {len(gt_trajectories)} missions.')

Loaded 1 missions.


------------------------------------------------------

In [19]:
max_diff = 10

delta = 10
delta_unit = metrics.Unit.frames
all_pairs = False

In [17]:
def plot_evaluations_APE(data, ape_metric, ape_stats):
    # Trajectory plot
    fig = plt.figure(figsize=(8, 6), dpi=160)
    traj_by_label = {
        "estimate": data[1],
        "reference": data[0]
    }
    plot.trajectories(fig, traj_by_label, plot.PlotMode.xy)
    plt.show()

    # Trajectory plot
    seconds_from_start = [t - data[1].timestamps[0] for t in data[1].timestamps]
    fig = plt.figure(figsize=(8, 6), dpi=160)
    plot.error_array(fig.gca(), ape_metric.error, x_array=seconds_from_start,
                     statistics={s:v for s,v in ape_stats.items() if s != "sse"},
                     name="APE", title="APE w.r.t. " + ape_metric.pose_relation.value, xlabel="$t$ (s)")
    plt.show()

    plot_mode = plot.PlotMode.xy
    fig = plt.figure(figsize=(8, 6), dpi=160)
    ax = plot.prepare_axis(fig, plot_mode)
    plot.traj(ax, plot_mode, gt_traj, '--', "gray", "reference")
    plot.traj_colormap(ax, data[1], ape_metric.error, 
                       plot_mode, min_map=ape_stats["min"], max_map=ape_stats["max"])
    ax.legend()
    plt.show()

def perform_evaluation_using_data(data, delta, delta_unit, all_pairs):
    print('--- Translational Part -----------------------------')
    pose_relation = metrics.PoseRelation.translation_part
    print('APE:')
    ape_metric_trans = metrics.APE(pose_relation)
    ape_metric_trans.process_data(data)
    ape_stat_trans = ape_metric_trans.get_statistic(metrics.StatisticsType.rmse)
    ape_stats_trans = ape_metric_trans.get_all_statistics()
    pprint.pprint(ape_stats_trans)

    print('RPE:')
    rpe_metric_trans = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
    rpe_metric_trans.process_data(data)
    rpe_stat_trans = rpe_metric_trans.get_statistic(metrics.StatisticsType.rmse)
    rpe_stats_trans = rpe_metric_trans.get_all_statistics()
    pprint.pprint(rpe_stats_trans)
    print('\n\n')

    print('--- Rotational Part -----------------------------')
    pose_relation = metrics.PoseRelation.rotation_angle_deg
    print('APE:')
    ape_metric_rot = metrics.APE(pose_relation)
    ape_metric_rot.process_data(data)
    ape_stat_rot = ape_metric_rot.get_statistic(metrics.StatisticsType.rmse)
    ape_stats_rot = ape_metric_rot.get_all_statistics()
    pprint.pprint(ape_stats_rot)

    print('RPE:')
    rpe_metric_rot = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
    rpe_metric_rot.process_data(data)
    rpe_stat_rot = rpe_metric_rot.get_statistic(metrics.StatisticsType.rmse)
    rpe_stats_rot = rpe_metric_rot.get_all_statistics()
    pprint.pprint(rpe_stats_rot)

    plot_evaluations_APE(data, ape_metric_trans, ape_stats_trans)

In [27]:
def ts_ns_to_seconds(ts_ns):
    k_ns_per_s = 1e9;
    return ts_ns / k_ns_per_s;

def convert_est_to_traj(trajectory):
    ts = ts_ns_to_seconds(trajectory[:,0])
    xyz = trajectory[:,1:4]
    wxyz = trajectory[:,4:8]
    return PoseTrajectory3D(positions_xyz = xyz, orientations_quat_wxyz = wxyz, timestamps = ts)

def convert_gt_to_traj(trajectory):
    ts = trajectory[:,0]
    xyz = trajectory[:,1:4]
    wxyz = trajectory[:,4:8]
    return PoseTrajectory3D(positions_xyz = xyz, orientations_quat_wxyz = wxyz, timestamps = ts)

idx = 0

print(f'=== Maplab ===========================================================')
est_traj = convert_est_to_traj(est_trajectories[idx])
gt_traj = convert_gt_to_traj(gt_trajectories[idx])
gt_traj, est_traj = sync.associate_trajectories(gt_traj, est_traj, max_diff)
est_traj.align(gt_traj, correct_scale=False, correct_only_scale=False)

data = (gt_traj, est_traj)
perform_evaluation_using_data(data, delta, delta_unit, all_pairs)

=== Maplab ===========================================================
--- Translational Part -----------------------------
APE:
{'max': 0.07169184771696438,
 'mean': 0.030891600201110876,
 'median': 0.029309943754202126,
 'min': 0.002435292561509256,
 'rmse': 0.03424856981945188,
 'sse': 1.5799832282110913,
 'std': 0.01478761548366053}
RPE:
{'max': 0.0338321763732913,
 'mean': 0.006790907803789793,
 'median': 0.006502383766958317,
 'min': 0.0003025169415477421,
 'rmse': 0.009064891917099875,
 'sse': 0.011011083572806158,
 'std': 0.006004651252914658}



--- Rotational Part -----------------------------
APE:
{'max': 2.9146563012794595,
 'mean': 0.29159775120868925,
 'median': 0.2571484173170258,
 'min': 0.010137960602061227,
 'rmse': 0.3976344703435531,
 'sse': 212.97844269127108,
 'std': 0.27034038450707537}
RPE:
{'max': 3.1299755346314546,
 'mean': 0.2764057602219023,
 'median': 0.12019843052288005,
 'min': 0.004274006837371153,
 'rmse': 0.5641993525870203,
 'sse': 42.65500186758811,

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:941: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…